In [2]:
import pandas as pd

# Check zillow_X_preprocessed.csv
X_path = r"D:\shri\RealtyAI\.venv\data\zillow\zillow_X_preprocessed.csv"
y_path = r"D:\shri\RealtyAI\.venv\data\zillow\zillow_y_preprocessed.csv"

try:
    X = pd.read_csv(X_path)
    y = pd.read_csv(y_path)
    
    print("✅ zillow_X_preprocessed.csv shape:", X.shape)
    print("✅ Columns in X (first 10):", X.columns.tolist()[:10])
    print("\n✅ zillow_y_preprocessed.csv shape:", y.shape)
    print("✅ Target column(s):", y.columns.tolist())
    
except Exception as e:
    print("❌ Error:", e)

✅ zillow_X_preprocessed.csv shape: (3563030, 32)
✅ Columns in X (first 10): ['MedianListingPrice_3Bedroom', 'ZHVI_AllHomes', 'Sale_Prices', 'MedianRentalPrice_AllHomes', 'PriceToRentRatio_AllHomes', 'Sale_Counts', 'PctOfHomesIncreasingInValues_AllHomes', 'PctOfListingsWithPriceReductions_AllHomes', 'ZHVI_SingleFamilyResidence', 'ZHVI_CondoCoop']

✅ zillow_y_preprocessed.csv shape: (3563030, 1)
✅ Target column(s): ['MedianListingPrice_AllHomes']


In [3]:
import pandas as pd

# Load original Zillow data
original_path = r"D:\shri\RealtyAI\.venv\data\zillow\City_time_series.csv"
original = pd.read_csv(original_path)

# Extract State from RegionName (last 2 letters)
def extract_state(name):
    if isinstance(name, str) and len(name) >= 2:
        suffix = name[-2:].upper()
        if suffix in ['CA', 'TX', 'FL']:
            return suffix
    return None

original['State'] = original['RegionName'].apply(extract_state)

# Keep only CA, TX, FL
filtered_original = original[original['State'].isin(['CA', 'TX', 'FL'])].copy()
filtered_original['Date'] = pd.to_datetime(filtered_original['Date'])

print("✅ Filtered original data shape:", filtered_original.shape)
print("✅ Regions:", filtered_original['State'].value_counts().to_dict())

✅ Filtered original data shape: (524798, 82)
✅ Regions: {'CA': 202308, 'TX': 171272, 'FL': 151218}


In [4]:
import pandas as pd
import numpy as np

# Use the filtered_original from Step 2
df = filtered_original.copy()

# Remove duplicates
print("Before removing duplicates:", df.shape)
df = df.drop_duplicates()
print("After removing duplicates:", df.shape)

# Handle missing values
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Fill numeric with median
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

# Fill categorical with mode
for col in categorical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0]) if not df[col].mode().empty else df[col].fillna("Unknown")

print("✅ Nulls after cleaning:", df.isnull().sum().sum())

# Save final dataset
output_path = r"D:\shri\RealtyAI\.venv\data\zillow\zillow_final_for_streamlit.csv"
df.to_csv(output_path, index=False)
print(f"✅ Saved final dataset to: {output_path}")
print("✅ Final shape:", df.shape)

Before removing duplicates: (524798, 82)
After removing duplicates: (524798, 82)


d:\shri\RealtyAI\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\shri\RealtyAI\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


✅ Nulls after cleaning: 1049596
✅ Saved final dataset to: D:\shri\RealtyAI\.venv\data\zillow\zillow_final_for_streamlit.csv
✅ Final shape: (524798, 82)


In [5]:
import pandas as pd
from prophet import Prophet
from prophet.serialize import model_to_json
import os

# Load final dataset
df = pd.read_csv(r"D:\shri\RealtyAI\.venv\data\zillow\zillow_final_for_streamlit.csv")
df['ds'] = pd.to_datetime(df['Date'])

# Create models directory
os.makedirs(r"D:\shri\RealtyAI\.venv\models\with_regressors", exist_ok=True)

# 30 Most Relevant Features (excluding target ZHVI_AllHomes)
key_features = [
    'ZHVI_SingleFamilyResidence',
    'ZHVI_CondoCoop',
    'ZHVI_BottomTier',
    'ZHVI_MiddleTier',
    'ZHVI_TopTier',
    'MedianListingPrice_AllHomes',
    'MedianListingPrice_SingleFamilyResidence',
    'MedianListingPrice_CondoCoop',
    'MedianRentalPrice_AllHomes',
    'MedianRentalPrice_SingleFamilyResidence',
    'ZRI_AllHomes',
    'ZRI_AllHomesPlusMultifamily',
    'ZriPerSqft_AllHomes',
    'PriceToRentRatio_AllHomes',
    'Sale_Prices',
    'Sale_Counts',
    'InventoryRaw_AllHomes',
    'InventorySeasonallyAdjusted_AllHomes',
    'PctOfHomesSellingForGain_AllHomes',
    'PctOfHomesSellingForLoss_AllHomes',
    'PctOfHomesIncreasingInValues_AllHomes',
    'PctOfHomesDecreasingInValues_AllHomes',
    'PctOfListingsWithPriceReductions_AllHomes',
    'MedianPctOfPriceReduction_AllHomes',
    'MedianPriceCutDollar_AllHomes',
    'ZHVI_1bedroom',
    'ZHVI_2bedroom',
    'ZHVI_3bedroom',
    'ZHVI_4bedroom'
]

# Verify features exist in dataset
available_features = [col for col in key_features if col in df.columns]
print(f"✅ Using {len(available_features)} regressors:", available_features)

regions = ['CA', 'TX', 'FL']

for region in regions:
    print(f"\nTraining {region} model with {len(available_features)} features...")
    
    # Filter and prepare data
    region_data = df[df['State'] == region].copy()
    region_data = region_data.rename(columns={'ZHVI_AllHomes': 'y'})  # ✅ TARGET = ZHVI_AllHomes
    
    # Keep only needed columns
    model_cols = ['ds', 'y'] + available_features
    model_data = region_data[model_cols].copy()
    model_data = model_data.fillna(0)  # Handle nulls safely
    
    if len(model_data) == 0:
        print(f"⚠️ No data for {region}")
        continue
    
    # Initialize Prophet
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05
    )
    
    # Add regressors
    for col in available_features:
        model.add_regressor(col)
    
    # Fit model
    model.fit(model_data)
    
    # Save model
    model_path = rf"D:\shri\RealtyAI\.venv\models\with_regressors\prophet_{region}_30features.json"
    with open(model_path, 'w') as fout:
        fout.write(model_to_json(model))
    
    print(f"✅ {region} model saved")

print("\n🎉 All models trained successfully!")

d:\shri\RealtyAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


✅ Using 29 regressors: ['ZHVI_SingleFamilyResidence', 'ZHVI_CondoCoop', 'ZHVI_BottomTier', 'ZHVI_MiddleTier', 'ZHVI_TopTier', 'MedianListingPrice_AllHomes', 'MedianListingPrice_SingleFamilyResidence', 'MedianListingPrice_CondoCoop', 'MedianRentalPrice_AllHomes', 'MedianRentalPrice_SingleFamilyResidence', 'ZRI_AllHomes', 'ZRI_AllHomesPlusMultifamily', 'ZriPerSqft_AllHomes', 'PriceToRentRatio_AllHomes', 'Sale_Prices', 'Sale_Counts', 'InventoryRaw_AllHomes', 'InventorySeasonallyAdjusted_AllHomes', 'PctOfHomesSellingForGain_AllHomes', 'PctOfHomesSellingForLoss_AllHomes', 'PctOfHomesIncreasingInValues_AllHomes', 'PctOfHomesDecreasingInValues_AllHomes', 'PctOfListingsWithPriceReductions_AllHomes', 'MedianPctOfPriceReduction_AllHomes', 'MedianPriceCutDollar_AllHomes', 'ZHVI_1bedroom', 'ZHVI_2bedroom', 'ZHVI_3bedroom', 'ZHVI_4bedroom']

Training CA model with 29 features...


23:14:47 - cmdstanpy - INFO - Chain [1] start processing
23:21:53 - cmdstanpy - INFO - Chain [1] done processing


✅ CA model saved

Training TX model with 29 features...


23:22:46 - cmdstanpy - INFO - Chain [1] start processing
23:23:50 - cmdstanpy - INFO - Chain [1] done processing


✅ TX model saved

Training FL model with 29 features...


23:24:32 - cmdstanpy - INFO - Chain [1] start processing
23:27:51 - cmdstanpy - INFO - Chain [1] done processing


✅ FL model saved

🎉 All models trained successfully!


In [9]:
import pandas as pd

# Load original Zillow data (full dataset)
original = pd.read_csv(r"D:\shri\RealtyAI\.venv\data\zillow\City_time_series.csv")

# Extract State from RegionName
def extract_state(name):
    if isinstance(name, str) and len(name) >= 2:
        suffix = name[-2:].upper()
        if suffix in ['CA', 'TX', 'FL']:
            return suffix
    return None

original['State'] = original['RegionName'].apply(extract_state)
filtered = original[original['State'].isin(['CA', 'TX', 'FL'])].copy()
filtered['Date'] = pd.to_datetime(filtered['Date'])

# Keep data up to 2017 (for holdout evaluation)
filtered = filtered[filtered['Date'] <= '2017-12-31']

# Handle nulls
numeric_cols = filtered.select_dtypes(include=['number']).columns
filtered[numeric_cols] = filtered[numeric_cols].fillna(filtered[numeric_cols].median())

categorical_cols = filtered.select_dtypes(exclude=['number']).columns
for col in categorical_cols:
    if filtered[col].isnull().any():
        filtered[col] = filtered[col].fillna(filtered[col].mode()[0] if not filtered[col].mode().empty else 'Unknown')

# Save
output_path = r"D:\shri\RealtyAI\.venv\data\zillow\zillow_final_with_2017.csv"
filtered.to_csv(output_path, index=False)
print(f"✅ Saved dataset with 2017 data to: {output_path}")
print("Shape:", filtered.shape)

✅ Saved dataset with 2017 data to: D:\shri\RealtyAI\.venv\data\zillow\zillow_final_with_2017.csv
Shape: (524798, 82)


In [ ]:
import pandas as pd
from prophet.serialize import model_from_json
import numpy as np
import matplotlib.pyplot as plt

# Load your final dataset
df = pd.read_csv(r"D:\shri\RealtyAI\.venv\data\zillow\zillow_final_with_2017.csv")
df['ds'] = pd.to_datetime(df['Date'])
df = df.sort_values('ds')

regions = ['CA', 'TX', 'FL']
target_col = 'ZHVI_AllHomes'

for region in regions:
    print(f"\n{'='*50}")
    print(f"🔍 EVALUATING {region} MODEL")
    print(f"{'='*50}")
    
    try:
        # Load model
        model_path = rf"D:\shri\RealtyAI\.venv\models\with_regressors\prophet_{region}_30features.json"
        with open(model_path, "r") as f:
            model = model_from_json(f.read())
        
        # Filter data
        region_data = df[df['State'] == region].copy()
        region_data = region_data.rename(columns={target_col: 'y'})
        region_data['y'] = pd.to_numeric(region_data['y'], errors='coerce')
        region_data = region_data.dropna(subset=['y', 'ds'])
        region_data = region_data.sort_values('ds')
        
        if len(region_data) == 0:
            print(f"⚠️ No valid data for {region}")
            continue
        
        # Use ALL available 2017 data (even if <12 months)
        test = region_data[(region_data['ds'] >= '2017-01-01') & (region_data['ds'] <= '2017-12-31')]
        
        if len(test) == 0:
            # Fallback: use last N months (whatever is available)
            test = region_data.tail(6)  # Use last 6 months if 2017 is empty
            if len(test) == 0:
                print(f"⚠️ No test data for {region}")
                continue
        
        # Create future dataframe for test period
future = test[['ds']].copy()

# 🔑 CRITICAL FIX: Ensure ds is datetime
future['ds'] = pd.to_datetime(future['ds'], unit='ns', errors='coerce')

# Get regressor columns
regressor_cols = [col for col in model.train_component_cols.keys() 
                 if col not in ['ds', 'y', 'trend', 'seasonal', 'additive_terms', 'multiplicative_terms']]

# Add regressors
for col in regressor_cols:
    if col in region_data.columns and len(region_data) > 0:
        future[col] = region_data[col].iloc[-1]
    else:
        future[col] = 0

future = future.apply(pd.to_numeric, errors='coerce').fillna(0)
        
        # Predict
        forecast = model.predict(future)
        
        # Merge
        test_merged = pd.merge(test[['ds', 'y']], forecast[['ds', 'yhat']], on='ds', how='inner')
        
        if len(test_merged) == 0:
            print(f"⚠️ No overlap for {region}")
            continue
        
        # Metrics
        y_true = test_merged['y']
        y_pred = test_merged['yhat']
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
        
        print(f"✅ MAPE: {mape:.2f}%")
        print(f"✅ RMSE: ${rmse:,.0f} (on {len(test_merged)} months)")
        
        # Plot
        plt.figure(figsize=(10, 4))
        plt.plot(test_merged['ds'], y_true, 'o-', label='Actual', color='blue')
        plt.plot(test_merged['ds'], y_pred, 'o-', label='Predicted', color='red')
        plt.title(f"{region} - Actual vs Predicted (Holdout)")
        plt.xlabel("Date")
        plt.ylabel("ZHVI")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Error: {e}")


🔍 EVALUATING CA MODEL


d:\shri\RealtyAI\.venv\Lib\site-packages\prophet\forecaster.py:290: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ds'] = pd.to_datetime(df['ds'])


❌ Error: Parsing "1485820800000000000" to datetime overflows, at position 0

🔍 EVALUATING TX MODEL


d:\shri\RealtyAI\.venv\Lib\site-packages\prophet\forecaster.py:290: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ds'] = pd.to_datetime(df['ds'])


❌ Error: Parsing "1485820800000000000" to datetime overflows, at position 0

🔍 EVALUATING FL MODEL
❌ Error: Parsing "1485820800000000000" to datetime overflows, at position 0


d:\shri\RealtyAI\.venv\Lib\site-packages\prophet\forecaster.py:290: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ds'] = pd.to_datetime(df['ds'])
